# A Basic Extract, Transform and Load (ETL) pipeline using Web Scrapping, Pandas and SQLite

#### Import necessary Libraries

In [ ]:
# icecream runs in Python Terminal with icecream installed in environment for complex code debugging
# Uncomment and Run Command Below:
!pip install icecream 
!pip install bs4
!pip install requests
!pip install pandas
!pip install lxml

  Attempting uninstall: executing
    Found existing installation: executing 0.8.3
    Uninstalling executing-0.8.3:
      Successfully uninstalled executing-0.8.3


In [ ]:
from io import StringIO # Extract
import requests # Extract
from bs4 import BeautifulSoup # Extract
import pandas as pd # Transformation
import sqlite3 # Load Database
from datetime import datetime # For Time of Log
from icecream import ic # We can skip icecream in Jupiter Notebook

#### Step 0: Maintaining a Log File
This step is done to record the logs while performing ETL and it is not neccessary in an ETL Pipeline.

In [ ]:
def log_progress(message):
    """This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing"""

    with open('./logs/code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

#### Step 1: Extract

In [ ]:
def extract(url, table_attribs):
    """ This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. """

    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Step 1: Data extraction complete. Data Extracted Successfully \
from Wikipedia. Initiating Transformation process')

    return df

#### Step 2: Transform

In [ ]:
def transform(df, csv_path):
    """ This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies"""

    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)

    log_progress('Step 2: Data transformation complete. \
Initiating Loading process')

    return df

#### Step 3: Load

##### Loading data to a CSV

In [ ]:
def load_to_csv(df, output_path):
    """ This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing."""

    df.to_csv(output_path)

    log_progress('Step 3.1: Data loaded to CSV file')

##### Loading data to SQL and Function to Run queries on that Database

In [ ]:
def load_to_db(df, sql_connection, table_name):
    """ This function saves the final data frame to a database
    table with the provided name. Function returns nothing."""

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Step 3.2: Data loaded to SQLite Database as a table, \
Executing queries')


# Query Function to Run queries on Database
def run_query(query_statement, sql_connection):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """

    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress(f'Process Complete. \
Query: {query_statement} Completed Successfully')

    return result


### Executing Pipeline

In [ ]:
if __name__ == '__main__':
    
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './output/largest_banks_data.csv'
    database_name = './output/Banks.db'
    table_name = 'largest_banks'
    
    log_progress('Step 0: Preliminaries complete. Initiating ETL process')

    
    df = extract(url, 'By market capitalization')
  

    transform(df, './input/exchange_rate.csv')
    

    load_to_csv(df, output_csv_path)

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)

        run_query('SELECT * FROM Largest_banks', conn)

        run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)

        run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn)
        log_progress('Query run')
    print(df)

In [ ]:
# Printing Queries to check data (in place of icecream) in Jupyter Notebook
query_1 = run_query('SELECT * FROM Largest_banks', conn)

print(query_1)

In [ ]:
query_2 = run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)

print(query_2)

In [ ]:
query_3 = run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn)

print(query_3)